In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import os


In [ ]:
data = pd.read_csv("/home/john/ds5030/understanding_uncertainty/data/ames_prices.csv")
data.head(10)

In [ ]:
# Example 1: create a DataFrame with a categorical column and show categories
df = pd.DataFrame({'category': ['A', 'B', 'C', 'D', 'E', 'F']})
# convert to categorical dtype (required to use .cat)
df['category'] = df['category'].astype('category')
print('Example categories for df:', df['category'].cat.categories)

# Example 2: show categories for a real dataset column if present
if 'data' in globals():
    col = 'MS Zoning'  # change to any categorical column name in your dataset
    if col in data.columns:
        data[col] = data[col].astype('category')
        print('\nCategories for', col, ':', data[col].cat.categories)
    else:
        print('\nColumn', col, 'not found in data; available columns:', list(data.columns))

In [ ]:
if 'data' in globals():
    col1 = 'Fireplaces'  # change to any categorical column name in your dataset
    col2 = 'Pool.Area'  # change to another categorical column name in your dataset
    if col1 in data.columns and col2 in data.columns:
        contingency_table = pd.crosstab(data[col1], data[col2])
        print('\nContingency table between', col1, 'and', col2, ':\n', contingency_table)

In [ ]:
data[['Fireplaces', 'Pool.Area']].isnull().sum()
#summarize the contingency table in percentages.
if 'data' in globals():
    col1 = 'Fireplaces'  # change to any categorical column name in your dataset
    col2 = 'Pool.Area'  # change to another categorical column name in your dataset
    if col1 in data.columns and col2 in data.columns:
        contingency_table = pd.crosstab(data[col1], data[col2], normalize='index') * 100
        print('\nContingency table (in percentages) between', col1, 'and', col2, ':\n', contingency_table)

## Interesting Patterns

I chose the Ames housing data because it has clean data for both of the variables I chose: fireplaces and pools as expressed in pool area. I summarized the data as percentages and found it interesting that 100% of the houses with four fireplaces, the largest number, also have the largest number sized pool are of 800 square feet. I have to believe this is the head coach's house of the Iowa State football team resident in Ames. Also, most people in Ames don't have pools until they have three fireplaces. Makes sense that larger, more well appointed houses would be the ones to have pools.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Load the data
df = pd.read_csv('nhanes_data_17_18.csv')

# Define our variables
Y = 'FeelingBadAboutYourself'  # Numeric dependent variable
X = 'EverUsedHeroin'           # Categorical independent variable

# Clean the data - remove missing values for both variables
clean_df = df[[X, Y]].dropna()

print("="*60)
print("NHANES ANALYSIS: FeelingBadAboutYourself by EverUsedHeroin")
print("="*60)
print(f"\nOriginal dataset size: {len(df):,} observations")
print(f"Complete cases for analysis: {len(clean_df):,} observations")
print(f"Missing data removed: {len(df) - len(clean_df):,} observations")

# Convert EverUsedHeroin to categorical labels for better interpretation
# Assuming typical NHANES coding: 1=Yes, 2=No
clean_df['Heroin_Use'] = clean_df[X].map({1: 'Yes', 2: 'No'})
clean_df['Heroin_Use'] = clean_df['Heroin_Use'].fillna('Unknown')

print(f"\nHeroin Use Categories:")
print(clean_df['Heroin_Use'].value_counts())

# 1. DESCRIPTIVE STATISTICS TABLE
print("\n" + "="*60)
print("1. DESCRIPTIVE STATISTICS TABLE")
print("="*60)

# Overall statistics
overall_stats = clean_df[Y].describe()
print(f"\nOverall Statistics for {Y}:")
print(overall_stats.round(3))

# Grouped statistics
grouped_stats = clean_df.groupby('Heroin_Use')[Y].describe()
print(f"\nGrouped Statistics for {Y} by Heroin Use:")
print(grouped_stats.round(3))

# Additional statistics
def additional_stats(series):
    return pd.Series({
        'skewness': stats.skew(series),
        'kurtosis': stats.kurtosis(series),
        'variance': np.var(series, ddof=1),
        'iqr': np.percentile(series, 75) - np.percentile(series, 25),
        'median_abs_dev': np.median(np.abs(series - np.median(series)))
    })

print(f"\nAdditional Statistics by Heroin Use:")
additional = clean_df.groupby('Heroin_Use')[Y].apply(additional_stats)
print(additional.round(3))

# 2. FREQUENCY TABLES
print("\n" + "="*60)
print("2. FREQUENCY TABLES")
print("="*60)

# Cross-tabulation
crosstab = pd.crosstab(clean_df[Y], clean_df['Heroin_Use'], margins=True)
print(f"\nCross-tabulation: {Y} by Heroin Use")
print(crosstab)

# Proportions within each heroin use category
prop_table = pd.crosstab(clean_df[Y], clean_df['Heroin_Use'], normalize='columns')
print(f"\nColumn Proportions (within Heroin Use category):")
print(prop_table.round(3))

# 3. STATISTICAL TESTS
print("\n" + "="*60)
print("3. STATISTICAL TESTS")
print("="*60)

# T-test (if we have Yes/No groups)
groups = clean_df.groupby('Heroin_Use')[Y].apply(list)
if len(groups) == 2:
    group1, group2 = groups.iloc[0], groups.iloc[1]
    t_stat, p_value = stats.ttest_ind(group1, group2)
    print(f"\nIndependent t-test:")
    print(f"t-statistic: {t_stat:.4f}")
    print(f"p-value: {p_value:.4f}")
    
# Mann-Whitney U test (non-parametric alternative)
if len(groups) == 2:
    u_stat, u_p_value = stats.mannwhitneyu(group1, group2, alternative='two-sided')
    print(f"\nMann-Whitney U test (non-parametric):")
    print(f"U-statistic: {u_stat:.4f}")
    print(f"p-value: {u_p_value:.4f}")

# ANOVA (if more than 2 groups)
if len(groups) > 2:
    f_stat, f_p_value = stats.f_oneway(*groups)
    print(f"\nOne-way ANOVA:")
    print(f"F-statistic: {f_stat:.4f}")
    print(f"p-value: {f_p_value:.4f}")

# 4. VISUALIZATIONS
print("\n" + "="*60)
print("4. CREATING VISUALIZATIONS")
print("="*60)

# Set up the plotting style
plt.style.use('default')
sns.set_palette("husl")

# Create figure with subplots
fig = plt.figure(figsize=(16, 12))

# 1. Kernel Density Plots
ax1 = plt.subplot(2, 3, 1)
for group_name, group_data in clean_df.groupby('Heroin_Use'):
    if len(group_data) > 1:  # Only plot if we have enough data
        sns.kdeplot(data=group_data[Y], label=f'{group_name} (n={len(group_data)})', 
                   fill=True, alpha=0.6)
plt.xlabel('Feeling Bad About Yourself Score')
plt.ylabel('Density')
plt.title('Kernel Density Plot by Heroin Use')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. Box Plot
ax2 = plt.subplot(2, 3, 2)
sns.boxplot(data=clean_df, x='Heroin_Use', y=Y)
plt.title('Box Plot by Heroin Use')
plt.ylabel('Feeling Bad About Yourself Score')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 3. Violin Plot
ax3 = plt.subplot(2, 3, 3)
sns.violinplot(data=clean_df, x='Heroin_Use', y=Y)
plt.title('Violin Plot by Heroin Use')
plt.ylabel('Feeling Bad About Yourself Score')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 4. Histogram with overlay
ax4 = plt.subplot(2, 3, 4)
for group_name, group_data in clean_df.groupby('Heroin_Use'):
    plt.hist(group_data[Y], alpha=0.6, label=f'{group_name} (n={len(group_data)})', 
             bins=np.arange(clean_df[Y].min(), clean_df[Y].max()+2)-0.5, 
             density=True)
plt.xlabel('Feeling Bad About Yourself Score')
plt.ylabel('Density')
plt.title('Histogram by Heroin Use')
plt.legend()
plt.grid(True, alpha=0.3)

# 5. Strip Plot
ax5 = plt.subplot(2, 3, 5)
sns.stripplot(data=clean_df, x='Heroin_Use', y=Y, size=4, alpha=0.6)
plt.title('Strip Plot by Heroin Use')
plt.ylabel('Feeling Bad About Yourself Score')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 6. Summary statistics bar plot
ax6 = plt.subplot(2, 3, 6)
means = clean_df.groupby('Heroin_Use')[Y].mean()
stds = clean_df.groupby('Heroin_Use')[Y].std()
x_pos = range(len(means))
plt.bar(x_pos, means, yerr=stds, capsize=5, alpha=0.7)
plt.xlabel('Heroin Use')
plt.ylabel('Mean Feeling Bad About Yourself Score')
plt.title('Mean ± SD by Heroin Use')
plt.xticks(x_pos, means.index, rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 5. DETAILED KERNEL DENSITY PLOT (SEPARATE FIGURE)
print("\nCreating detailed kernel density plot...")

plt.figure(figsize=(12, 8))

# Create more detailed KDE plot
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
for i, (group_name, group_data) in enumerate(clean_df.groupby('Heroin_Use')):
    if len(group_data) > 1:
        # Calculate KDE
        kde_data = group_data[Y].dropna()
        density = stats.gaussian_kde(kde_data)
        xs = np.linspace(clean_df[Y].min(), clean_df[Y].max(), 200)
        density_values = density(xs)
        
        # Plot
        plt.fill_between(xs, density_values, alpha=0.4, color=colors[i % len(colors)], 
                        label=f'{group_name} (n={len(kde_data)})')
        plt.plot(xs, density_values, color=colors[i % len(colors)], linewidth=2)

plt.xlabel('Feeling Bad About Yourself Score', fontsize=12)
plt.ylabel('Probability Density', fontsize=12)
plt.title('Kernel Density Estimation: Feeling Bad About Yourself by Heroin Use', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 6. SUMMARY TABLE FOR EXPORT
print("\n" + "="*60)
print("6. SUMMARY TABLE")
print("="*60)

# Create comprehensive summary table
summary_stats = []
for group_name, group_data in clean_df.groupby('Heroin_Use'):
    y_data = group_data[Y]
    stats_dict = {
        'Group': group_name,
        'N': len(y_data),
        'Mean': y_data.mean(),
        'Std': y_data.std(),
        'Min': y_data.min(),
        'Q1': y_data.quantile(0.25),
        'Median': y_data.median(),
        'Q3': y_data.quantile(0.75),
        'Max': y_data.max(),
        'Skewness': stats.skew(y_data),
        'Kurtosis': stats.kurtosis(y_data)
    }
    summary_stats.append(stats_dict)

summary_df = pd.DataFrame(summary_stats)
print("\nComprehensive Summary Statistics:")
print(summary_df.round(3))

# Save summary table
summary_df.to_csv('nhanes_summary_stats.csv', index=False)
print(f"\nSummary statistics saved to 'nhanes_summary_stats.csv'")

print("\n" + "="*60)
print("ANALYSIS COMPLETE")
print("="*60)